In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lrex').getOrCreate()
data  = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', inferSchema=True, header=True)

data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [2]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [4]:
data.columns

Out[33]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [5]:
asssembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website',  'Length of Membership'],
                            outputCol = 'features')

In [6]:
output = asssembler.transform(data)

In [7]:
output.head(1)

Out[40]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [8]:
final_data = output.select('features', 'Yearly Amount Spent')

In [9]:
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [10]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [11]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 337|
 mean| 497.7106210938881|
 stddev| 78.17494650790316|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [12]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [13]:
lr_model = lr.fit(train_data)

In [14]:
test_result = lr_model.evaluate(test_data)

In [15]:
test_result.residuals.show()

+-------------------+
 residuals|
+-------------------+
-12.289705019391135|
-17.987953025295724|
 6.864820357566487|
 -21.1331847805194|
 -6.837849067531124|
 10.239825144947417|
 22.79172830364496|
 2.7472035346282837|
 4.190577601626387|
-0.2554118961172662|
 -5.76110845593962|
 -9.648188216922733|
-16.966986099181668|
 17.963955258094984|
 16.991423337947992|
 6.548070150209185|
 -4.022771338720531|
 8.295333098335561|
-2.6661743485660736|
-10.344459210809987|
+-------------------+
only showing top 20 rows

In [16]:
test_result.rootMeanSquaredError

Out[50]: 10.800908880274614

In [17]:
test_result.r2

Out[52]: 0.9824428549242511

In [18]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [19]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[30.3931845423455...|
[30.8162006488763...|
[30.9716756438877...|
[31.1239743499119...|
[31.1280900496166...|
[31.1695067987115...|
[31.2834474760581...|
[31.3091926408918...|
[31.3584771924370...|
[31.3895854806643...|
[31.5147378578019...|
[31.5261978982398...|
[31.5702008293202...|
[31.6005122003032...|
[31.6098395733896...|
[31.6548096756927...|
[31.8124825597242...|
[31.8512531286083...|
[31.8530748017465...|
[31.8648325480987...|
+--------------------+
only showing top 20 rows

In [20]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.3931845423455...|332.21857482258474|
[30.8162006488763...| 284.0742939737647|
[30.9716756438877...|487.77378939932623|
[31.1239743499119...| 508.0802386202852|
[31.1280900496166...| 564.0905358145858|
[31.1695067987115...| 417.1167056573454|
[31.2834474760581...| 568.9893611220225|
[31.3091926408918...|429.97351430530534|
[31.3584771924370...| 490.985372847849|
[31.3895854806643...|410.32502295610016|
[31.5147378578019...|495.57359645240103|
[31.5261978982398...|418.74271440926054|
[31.5702008293202...| 562.9124782405866|
[31.6005122003032...|461.20889623300195|
[31.6098395733896...|427.55412631316017|
[31.6548096756927...|468.71535357733933|
[31.8124825597242...|396.83311632251775|
[31.8512531286083...|464.69691356846283|
[31.8530748017465...|461.95129781091805|
[31.8648325480987...|450.23573968762366|
+--------------------+------------------+
only showing top 20 rows